# **Estudo de Caso 1: ETL**

Este Jupyter Notebook demonstra um fluxo completo de **ETL** (Extract, Transform, Load) com as seguintes etapas principais:

---

## **1. Extração (Extract)**
Nesta etapa, os dados são extraídos de um dataset "sujo" disponibilizado no **Kaggle**. Essa fonte contém informações que precisam de limpeza e padronização para uso.

---

## **2. Transformação (Transform)**
O dataset passa por um processo de tratamento e transformação, incluindo:
- **Limpeza de dados**: Remoção de valores nulos, inconsistências e outliers.
- **Transformações estruturais**: Ajustes de formato e normalização de colunas.
- **Enriquecimento**: Inclusão de informações adicionais ou calculadas.

As transformações são realizadas com **Python**, utilizando bibliotecas como:
- `Pandas`: Para manipulação de dados em tabelas.
- `NumPy`: Para cálculos e operações numéricas.

---

## **3. Carregamento (Load)**
O dataset tratado é carregado em um banco de dados **PostgreSQL**, tornando os dados acessíveis para análises posteriores e consultas investigativas.

---

### **Ferramentas e Tecnologias Utilizadas**
- **Python**: Linguagem de programação para automação das etapas de ETL.
- **Pandas e NumPy**: Bibliotecas essenciais para manipulação e transformação de dados.
- **PostgreSQL**: Banco de dados relacional para armazenamento e consulta de dados limpos.
- **Jupyter Notebook**: Ambiente interativo para execução do código e visualização dos resultados.

---

### **Objetivo**
Este estudo de caso demonstra as etapas práticas de um fluxo ETL, preparando dados brutos para uso eficiente em análises e tomada de decisão.


## Extração e Transformação 

In [148]:
# import de libs necessárias para o processo de data cleaning

import numpy as np
import pandas as pd
import zipfile
import os
from datetime import datetime
import urllib.request
import json
import pandas as pd
import csv
import io

In [149]:
# armazenando os ids de cada arquivo que serão baixados posteriormente
files_ids = ["99b42b09-95af-47de-8411-ab99c380c3ef",
             "9664de94-9f07-4adc-848d-b6ef56510762",
             "bceb5759-5500-49db-bc86-b038892acc06",
             "ca7fb968-3a2c-44ff-a2e8-730d1a689407",
              "c675899c-69d9-4dc8-bb11-00afc9636a3b"]

In [150]:
# Diretório onde o dataset será salvo
dataset_dir = "datasets"
os.makedirs(dataset_dir, exist_ok=True)

try:
    # URL direta do arquivo JSON
    for fileId in files_ids:
        url = f'http://dados.recife.pe.gov.br/datastore/dump/{fileId}?format=json'
        # Nome do arquivo de saída
        output_path = os.path.join(dataset_dir, f"{fileId}.json")
        try:
            print("Baixando arquivo JSON...")
            urllib.request.urlretrieve(url, output_path)
            print(f"Arquivo salvo em: {output_path}")
        except Exception as e:
            print(f"Erro ao baixar o arquivo: {e}")
    print("Todos os downloads foram concluídos com sucesso!")
except Exception as e:
    pass


Baixando arquivo JSON...
Arquivo salvo em: datasets\99b42b09-95af-47de-8411-ab99c380c3ef.json
Baixando arquivo JSON...
Arquivo salvo em: datasets\9664de94-9f07-4adc-848d-b6ef56510762.json
Baixando arquivo JSON...
Arquivo salvo em: datasets\bceb5759-5500-49db-bc86-b038892acc06.json
Baixando arquivo JSON...
Arquivo salvo em: datasets\ca7fb968-3a2c-44ff-a2e8-730d1a689407.json
Baixando arquivo JSON...
Arquivo salvo em: datasets\c675899c-69d9-4dc8-bb11-00afc9636a3b.json
Todos os downloads foram concluídos com sucesso!


Antes de carregar os dados no pandas, precisamos normalizar o JSON, pois ele segue o formato da API CKAN (utilizada por portais de dados abertos, como o do Recife). Nesse formato, os dados vêm separados em duas partes:

"fields": define o nome e tipo de cada coluna.

"records": contém os registros em formato de lista de listas (sem nomes de colunas embutidos).

Por isso, é necessário extrair os nomes das colunas a partir de "fields" e aplicá-los aos dados de "records" ao construir o DataFrame. Isso garante que os dados fiquem organizados corretamente no pandas.

In [151]:
df = None
for fileName in files_ids:
    json_dir = f"datasets/{fileName}.json"
    # Abre e carrega o conteúdo do arquivo JSON em um dicionário Python
    with open(json_dir, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        
    # Extrai os nomes das colunas da lista "fields" e as tuplas da lista "records"
    tuples = json_data['records']
    cols = [field['id'] for field in json_data['fields']]
    
    # se for o primeiro json lido, vamos instanciar nosso dataFrame pandas que será usado no resto da aplicação
    if(fileName == files_ids[0]):
        df = pd.DataFrame(data=tuples, columns=cols)
    else:
        # caso contrário, vamos criar um dataframe temporário e concatenar com o já existente
        temp_df =  pd.DataFrame(data=tuples, columns=cols)
        df = pd.concat([df, temp_df], ignore_index=True)
        
print("Dataframe inicializado com sucesso!")
print(f"{len(files_ids)} jsons adicionados ao df")

Dataframe inicializado com sucesso!
5 jsons adicionados ao df


In [152]:
# Vamos dar uma olhadinha no dataframe pra ver se tá tudo certo
print(f"{df.shape[0]} linhas e {df.shape[1]} colunas")
df.head()

83996 linhas e 14 colunas


,_id,faixa_etaria,idade,sexo,raca_cor,municipio,grupo,categoria,lote,vacina_fabricante,descricao_dose,cnes,sistema_origem,data_vacinacao
0,1,50 a 54 anos,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,,FM3457,3 - COMIRNATY (PFIZER),3,DS 1: CNES: 000507 - POLICLÍNICA GOUVEIA DE BA...,Conecta Recife,2021-12-06T00:00:00
1,2,50 a 54 anos,54,MASCULINO,PARDA,JABOATÃO DOS GUARARAPES,TRABALHADORES DA EDUCAÇÃO,OUTROS,FM3802,3 - COMIRNATY (PFIZER),2,DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE,Conecta Recife,2021-12-15T00:00:00
2,3,50 a 54 anos,53,FEMININO,PARDA,RECIFE,TRABALHADORES DA EDUCAÇÃO,ENSINO BÁSICO ESTADUAL,FM3802,3 - COMIRNATY (PFIZER),3,DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER,Conecta Recife,2021-12-30T00:00:00
3,4,50 a 54 anos,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,,FM3457,3 - COMIRNATY (PFIZER),3,DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE,Conecta Recife,2021-12-17T00:00:00
4,5,50 a 54 anos,54,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL (18 a 59 anos),,FM3457,3 - COMIRNATY (PFIZER),1,DS 7: CNES: 0000647 - POLICLINICA CLEMENTINO F...,Conecta Recife,2021-12-05T00:00:00


In [153]:
# vamos olhar se o nome das colunas estão ok
df.columns

Index(['_id', 'faixa_etaria', 'idade', 'sexo', 'raca_cor', 'municipio',
       'grupo', 'categoria', 'lote', 'vacina_fabricante', 'descricao_dose',
       'cnes', 'sistema_origem', 'data_vacinacao'],
      dtype='object')

Em relação a nomeação, todas as colunas estão no padrão correto, não há nada que possamos melhorar

In [154]:
print(df.isna().sum()) # valores faltantes por coluna

duplicated_ids = df["_id"].duplicated().sum()
print(f"IDs duplicados: {duplicated_ids}") # total de duplicados

_id                  0
faixa_etaria         0
idade                0
sexo                 0
raca_cor             0
municipio            0
grupo                0
categoria            0
lote                 0
vacina_fabricante    0
descricao_dose       0
cnes                 0
sistema_origem       0
data_vacinacao       0
dtype: int64
IDs duplicados: 51996


Como juntamos todas as bases de dados, a coluna _id foi prejudicada, pois todos os 5 arquivos tinha um id ascendente e incremental começando do 1. Ou seja, algumas tuplas possuem o mesmo ID após a junção.
Solução: precisamos remover a coluna _id e criarmos uma novamente (ascendente e incremental). Como essa chave primária não faz referência direta à tupla, isso não vai ser um problema (se fosse um CPF por exemplo, a solução seria outra)

In [155]:
#apagando coluna _id
df = df.drop('_id', axis=1)
# criando nova coluna ID
df['id'] = range(len(df))

# Reorganiza colocando 'id' como a primeira coluna
cols = ['id'] + [col for col in df.columns if col != 'id']
df = df[cols]
df.head()

,id,faixa_etaria,idade,sexo,raca_cor,municipio,grupo,categoria,lote,vacina_fabricante,descricao_dose,cnes,sistema_origem,data_vacinacao
0,0,50 a 54 anos,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,,FM3457,3 - COMIRNATY (PFIZER),3,DS 1: CNES: 000507 - POLICLÍNICA GOUVEIA DE BA...,Conecta Recife,2021-12-06T00:00:00
1,1,50 a 54 anos,54,MASCULINO,PARDA,JABOATÃO DOS GUARARAPES,TRABALHADORES DA EDUCAÇÃO,OUTROS,FM3802,3 - COMIRNATY (PFIZER),2,DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE,Conecta Recife,2021-12-15T00:00:00
2,2,50 a 54 anos,53,FEMININO,PARDA,RECIFE,TRABALHADORES DA EDUCAÇÃO,ENSINO BÁSICO ESTADUAL,FM3802,3 - COMIRNATY (PFIZER),3,DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER,Conecta Recife,2021-12-30T00:00:00
3,3,50 a 54 anos,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,,FM3457,3 - COMIRNATY (PFIZER),3,DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE,Conecta Recife,2021-12-17T00:00:00
4,4,50 a 54 anos,54,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL (18 a 59 anos),,FM3457,3 - COMIRNATY (PFIZER),1,DS 7: CNES: 0000647 - POLICLINICA CLEMENTINO F...,Conecta Recife,2021-12-05T00:00:00


Vamos olhar quantas colunas estão 50%+ vazias

In [156]:
lines_qnt = df.shape[0]
for category in df.columns:
    empty_row = df[category].eq('').sum()
    percentual_empty_row = (empty_row / lines_qnt) * 100
    # vou exibir apenas as colunas mais preocupantes (acima de 50% vazia)
    if(percentual_empty_row > 50):
        print(f"A coluna '{category}' está {percentual_empty_row:.2f}% vazia")

A coluna 'categoria' está 74.12% vazia


Como podemos ver, a coluna "categoria" está bem vazia (e as que não estão possuem apenas 'OUTROS' como valor), não conseguímos reaproveitar muito esses dados, pois são poucos comparados com nossa base de dados. Vamos excluir a coluna 'categoria':

In [157]:
#apagando coluna categoria
df = df.drop('categoria', axis=1)
df.head()

,id,faixa_etaria,idade,sexo,raca_cor,municipio,grupo,lote,vacina_fabricante,descricao_dose,cnes,sistema_origem,data_vacinacao
0,0,50 a 54 anos,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,FM3457,3 - COMIRNATY (PFIZER),3,DS 1: CNES: 000507 - POLICLÍNICA GOUVEIA DE BA...,Conecta Recife,2021-12-06T00:00:00
1,1,50 a 54 anos,54,MASCULINO,PARDA,JABOATÃO DOS GUARARAPES,TRABALHADORES DA EDUCAÇÃO,FM3802,3 - COMIRNATY (PFIZER),2,DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE,Conecta Recife,2021-12-15T00:00:00
2,2,50 a 54 anos,53,FEMININO,PARDA,RECIFE,TRABALHADORES DA EDUCAÇÃO,FM3802,3 - COMIRNATY (PFIZER),3,DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER,Conecta Recife,2021-12-30T00:00:00
3,3,50 a 54 anos,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,FM3457,3 - COMIRNATY (PFIZER),3,DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE,Conecta Recife,2021-12-17T00:00:00
4,4,50 a 54 anos,54,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL (18 a 59 anos),FM3457,3 - COMIRNATY (PFIZER),1,DS 7: CNES: 0000647 - POLICLINICA CLEMENTINO F...,Conecta Recife,2021-12-05T00:00:00


A coluna 'faixa etária' é um pouco inútil, pois ao saber a idade você sabe a faixa etária do indivíduo. Vamos removê-la

In [158]:
df = df.drop('faixa_etaria', axis=1)

A coluna cnes (Código do cadastro nacional de estabelecimento de saúde)
está seguindo o padrão do exemplo a seguir:
- "DS X: CNES: XXXXXXX - NOME DO ESTABELECIMENTO DE SAÚDE"

Algumas observações:
- O nome do estabelecimento e o DS é dependente de forma transittiva do CNES
- o DS aparentemente não tem uma utilidade aparente (deve ser apenas uma indexação no BD da prefeitura)

Conclusão:
- Precisamos criar um novo dataframe para o CNES, que terá apenas o CNES e o nome do estabelecimento;
- A coluna cnes será normalizada e passará a conter apenas o código cnes
- o valor de DS será omitido nos dois dataframes, pois não encontramos utilidade nele

In [159]:
cnes_df = pd.DataFrame(df['cnes'])

#utilizando regex para extrair e separar o código CNES e o nome da unidade de saúde em colunas
cnes_df[['cnes_codigo', 'unidade_saude_nome']] = cnes_df['cnes'].str.extract(
    r'CNES:\s*([0-9A-Z]+)\s*-\s*(.+)'
)

cnes_df = cnes_df.drop('cnes', axis=1)  # removendo a coluna original 'cnes'

#removendo duplicata
cnes_df = cnes_df.drop_duplicates(subset="cnes_codigo")

duplicated_cnes = cnes_df["cnes_codigo"].duplicated().sum()
print(f"CNES duplicados (se for 0, nossa operação deu certo): {duplicated_cnes}") # total de duplicados

#alterando a tabela df e deixando apenas o código cnes agora
df['cnes'] = df['cnes'].str.extract(r'CNES:\s*(\d{6,7})')
df.rename(columns={'cnes': 'cnes_codigo'}, inplace=True)
df.head()



CNES duplicados (se for 0, nossa operação deu certo): 0


,id,idade,sexo,raca_cor,municipio,grupo,lote,vacina_fabricante,descricao_dose,cnes_codigo,sistema_origem,data_vacinacao
0,0,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,FM3457,3 - COMIRNATY (PFIZER),3,000507,Conecta Recife,2021-12-06T00:00:00
1,1,54,MASCULINO,PARDA,JABOATÃO DOS GUARARAPES,TRABALHADORES DA EDUCAÇÃO,FM3802,3 - COMIRNATY (PFIZER),2,0000639,Conecta Recife,2021-12-15T00:00:00
2,2,53,FEMININO,PARDA,RECIFE,TRABALHADORES DA EDUCAÇÃO,FM3802,3 - COMIRNATY (PFIZER),3,6897029,Conecta Recife,2021-12-30T00:00:00
3,3,53,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,FM3457,3 - COMIRNATY (PFIZER),3,0000639,Conecta Recife,2021-12-17T00:00:00
4,4,54,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL (18 a 59 anos),FM3457,3 - COMIRNATY (PFIZER),1,0000647,Conecta Recife,2021-12-05T00:00:00


Com essa operação, temos uma nova tabela de cnes normalizada e a coluna cnes da tabela original não está mais com dados multivalorados 
(mais performance e manutenabilidade)